In [1]:
import os
import sys
import numpy as np
import ccdproc as ccdp
import astropy.units as u
#import numpy as np
#import os
from astropy import units as u
#import ccdproc as ccdp

#import os
#from astropy import units as u
#import ccdproc as ccdp

import matplotlib.pyplot as plt
from astropy.visualization import (PercentileInterval, ZScaleInterval, ImageNormalize)
from astropy.io import fits

#import matplotlib.pyplot as plt
#from astropy.visualization import (PercentileInterval, ZScaleInterval, ImageNormalize)
#from astropy.io import fits


# Import Functions.py file located in hrpo-pipeline
import sys
sys.path.append('/home/idies/workspace/Storage/treed28/persistent/hrpo-pipeline/')
import Functions as f

In [2]:
base = "/home/idies/workspace/"
data_dir = "Temporary/treed28/scratch/20240903"

#/home/idies/workspace/Temporary/treed28/scratch/20240903/reduced/

path = os.path.join(base, data_dir)
os.chdir(path)

os.path.exists

images = ccdp.ImageFileCollection(path)
images.summary

file,simple,bitpix,naxis,naxis1,naxis2,bscale,bzero,date-obs,exptime,exposure,set-temp,ccd-temp,xpixsz,ypixsz,xbinning,ybinning,xorgsubf,yorgsubf,readoutm,imagetyp,focuspos,focustem,object,objctra,objctdec,objctalt,objctaz,objctha,sitelat,sitelong,jd,jd-helio,airmass,focallen,aptdia,aptarea,swcreate,sbstdver,telescop,instrume,observer,notes,unit,flipstat,swowner,filter
str19,bool,int64,int64,int64,int64,float64,float64,str19,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,str10,str11,int64,float64,str10,str8,str9,str8,str8,str8,str8,str9,float64,float64,float64,float64,float64,float64,str34,str21,str30,str14,str1,str1,str3,str1,str23,object
Bias-001Bias.fit,True,16,2,1024,1024,1.0,32768.0,2024-09-04T03:36:24,0.0,0.0,-5.0,-4.8626077500000005,13.0,13.0,1,1,0,0,Monochrome,Bias Frame,8560,630.0,Bias,15 59 25,+25 55 17,31.8628,283.3599,4.4446,30 20 45,-91 04 13,2460557.6502777776,2460557.6487624766,1.8877121610835381,4115.0,508.0,200656.16730637552,MaxIm DL Version 5.24 140606 2EUKY,SBFITSEXT Version 1.0,Highland Road Park Observatory,Apogee USB/Net,,,adu,,LSU Physics-Astronomy-1,--
Bias-002Bias.fit,True,16,2,1024,1024,1.0,32768.0,2024-09-04T03:36:34,0.0,0.0,-5.0,-4.8626077500000005,13.0,13.0,1,1,0,0,Monochrome,Bias Frame,8560,630.0,Bias,15 59 25,+25 55 17,31.8165,283.3809,4.4482,30 20 45,-91 04 13,2460557.6503935186,2460557.64887821,1.8895566249126776,4115.0,508.0,200656.16730637552,MaxIm DL Version 5.24 140606 2EUKY,SBFITSEXT Version 1.0,Highland Road Park Observatory,Apogee USB/Net,,,adu,,LSU Physics-Astronomy-1,--
Bias-003Bias.fit,True,16,2,1024,1024,1.0,32768.0,2024-09-04T03:36:45,0.0,0.0,-5.0,-4.8626077500000005,13.0,13.0,1,1,0,0,Monochrome,Bias Frame,8560,630.0,Bias,15 59 25,+25 55 17,31.7888,283.3935,4.4504,30 20 45,-91 04 13,2460557.650520833,2460557.6490055164,1.8915903480000875,4115.0,508.0,200656.16730637552,MaxIm DL Version 5.24 140606 2EUKY,SBFITSEXT Version 1.0,Highland Road Park Observatory,Apogee USB/Net,,,adu,,LSU Physics-Astronomy-1,--
Bias-004Bias.fit,True,16,2,1024,1024,1.0,32768.0,2024-09-04T03:36:55,0.0,0.0,-5.0,-4.8626077500000005,13.0,13.0,1,1,0,0,Monochrome,Bias Frame,8560,630.0,Bias,15 59 25,+25 55 17,31.7521,283.4101,4.4534,30 20 45,-91 04 13,2460557.6506365743,2460557.64912125,1.8934435927242368,4115.0,508.0,200656.16730637552,MaxIm DL Version 5.24 140606 2EUKY,SBFITSEXT Version 1.0,Highland Road Park Observatory,Apogee USB/Net,,,adu,,LSU Physics-Astronomy-1,--
Bias-005Bias.fit,True,16,2,1024,1024,1.0,32768.0,2024-09-04T03:37:06,0.0,0.0,-5.0,-5.2743735,13.0,13.0,1,1,0,0,Monochrome,Bias Frame,8560,630.0,Bias,15 59 25,+25 55 17,31.7055,283.4312,4.4570,30 20 45,-91 04 13,2460557.650763889,2460557.6492485567,1.8954870060905855,4115.0,508.0,200656.16730637552,MaxIm DL Version 5.24 140606 2EUKY,SBFITSEXT Version 1.0,Highland Road Park Observatory,Apogee USB/Net,,,adu,,LSU Physics-Astronomy-1,--
Bias-006Bias.fit,True,16,2,1024,1024,1.0,32768.0,2024-09-04T03:37:16,0.0,0.0,-5.0,-5.2743735,13.0,13.0,1,1,0,0,Monochrome,Bias Frame,8560,630.0,Bias,15 59 25,+25 55 17,31.6714,283.4467,4.4598,30 20 45,-91 04 13,2460557.6508796294,2460557.64936429,1.8973490812597464,4115.0,508.0,200656.16730637552,MaxIm DL Version 5.24 140606 2EUKY,SBFITSEXT Version 1.0,Highland Road Park Observatory,Apogee USB/Net,,,adu,,LSU Physics-Astronomy-1,--
Bias-007Bias.fit,True,16,2,1024,1024,1.0,32768.0,2024-09-04T03:37:26,0.0,0.0,-5.0,-5.2743735,13.0,13.0,1,1,0,0,Monochrome,Bias Frame,8560,630.0,Bias,15 59 25,+25 55 17,31.6353,283.4631,4.4626,30 20 45,-91 04 13,2460557.6509953705,2460557.6494800234,1.8992153926895932,4115.0,508.0,200656.16730637552,MaxIm DL Version 5.24 140606 2EUKY,SBFITSEXT Version 1.0,Highland Road Park Observatory,Apogee USB/Net,,,adu,,LSU Physics-Astronomy-1,--
Bias-008Bias.fit,True,16,2,1024,1024,1.0,32768.0,2024-09-04T03:37:35,0.0,0.0,-5.0,-5.2743735,13.0,13.0,1,1,0,0,Monochrome,Bias Frame,8560,630.0,Bias,15 59 25,+25 55 17,31.6023,283.4781,4.4652,30 20 45,-91 04 13,2460557.651099537,2460557.649

In [3]:
# Filter images to get only the bias frames
biases = images.filter(imagetyp="Bias Frame")

# Create a list of CCDData objects from the filtered bias frames
bias_frames = [ccd for ccd in biases.ccds(ccd_kwargs={'unit': u.adu})]

# Combine the bias frames using the median method
mbias = ccdp.combine(bias_frames, method='median')

# Save the master bias frame, overwriting if necessary
mbias.write("reduced/mbias.fit", overwrite=True)

In [4]:
# Filter images to get only the dark frames
darks = images.filter(imagetyp="Dark Frame")

# Initialize the lists for reduced darks and exposure times
exptimes = []
red_darks = []

# Iterate over each dark frame, subtracting the master bias and storing results
#for ccd in darks.ccds(ccd_kwargs={'unit': u.adu}):

for ccd in darks.ccds(ccd_kwargs={'unit':u.adu}):    
    # Subtract the master bias from each dark frame
    #cleaned_dark = ccdp.subtract_bias(ccd, mbias)
    cleaned_dark = ccdp.subtract_bias(ccd, mbias)
    
    # Append the cleaned dark frame to the list
    red_darks.append(cleaned_dark)
    
    # Append the exposure time to the exptimes list
    exptimes.append(cleaned_dark.header["exptime"])
    
# Convert the exposure times list to a numpy array
exptimes = np.array(exptimes)

# Create master dark frame with `ccdp.combine`. Use the median method, set `scale=1/exptimes`, and set `unit="adu"`. I will refer to this as `mdark` 
mdark = ccdp.combine(red_darks, method='median', scale=1/exptimes, unit=u.adu) #unit=u.adu

# Set the `"exposure"` and `"exptime"` header keywords in `mdark.header` to 1
mdark.header["exptime"] = 1
mdark.header["exposure"] = 1

# Ensure the 'reduced' directory exists
output_directory = "reduced"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
# Save `mdark` in the reduced data directory
mdark.write(os.path.join(output_directory, "mdark.fit"), overwrite=True)
#mdark.write("reduced/mdark.fit", overwrite=True)

In [5]:
# Define a list of filter names
filters = ["Bessel B", "Bessel V", "Bessel R"]

# Create an empty dictionary to store the master flats for each filter
mflats = {}

# Ensure the 'reduced' directory exists
output_directory = "reduced"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate over the filters
for filt in filters:
    # Filter the images to get only the flat frames for the current filter
    flats = images.filter(imagetyp="Flat Field", filter=filt)
    
    # Create an empty list to store the reduced flats for the current filter
    red_flats = []
    
    # Iterate over the flat frames
    for flat in flats.ccds(ccd_kwargs={'unit': u.adu}):
        # Subtract the master bias from each flat
        flat_bias_subtracted = ccdp.subtract_bias(flat, mbias)
        
        # Subtract the master dark from each flat, with scaling
        flat_reduced = ccdp.subtract_dark(flat_bias_subtracted, mdark, 
                                          scale=True, 
                                          exposure_time='exptime', 
                                          exposure_unit=u.s)
        
        # Append the reduced flat to the red_flats list
        red_flats.append(flat_reduced)
    
    # Prepare a unique filename for the current filter
    filt_string = filt.replace(" ", "_")
    fname = os.path.join(output_directory, f"{filt_string}_flat.fit")
    
    # Combine the reduced flats into a master flat for the current filter
    mflat = ccdp.combine(red_flats, method='median', unit=u.adu)
    
    # Save the master flat to the reduced directory
    mflat.write(fname, overwrite=True)
    
    # Add the master flat to the dictionary with the filter name as the key
    mflats[filt] = mflat.copy()



In [6]:
# !!!Correct this cell to write the filenames correctly... 
# the reduced files should keep their original file name, include the filter (-001B, -001R, -001V), and add "reduced" or "light"

# Filter images to get only the light frames
lights = images.filter(imagetyp="Light Frame")

# Ensure the 'reduced' directory exists
output_directory = "reduced"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate over the light frames and reduce them
for light in lights.ccds(ccd_kwargs={'unit': u.adu}):
    # Reduce the light frame using ccd_process
    reduced_light = ccdp.ccd_process(
        light,
        master_bias=mbias,
        dark_frame=mdark,
        master_flat=mflats[light.header["Filter"]],
        dark_scale=True,
        exposure_key='exptime',
        exposure_unit=u.s
    )
    
    # Prepare the output filename
    #filename = os.path.join(output_directory, f"{light.header['object']}_reduced.fit")
    
    # Write out the reduced light frame to the reduced data directory
    #reduced_light.write(filename, overwrite=True)

# Extract the original filename without extension
    original_filename = os.path.splitext(output_directory, f"{light.filename})[0]
    
    # Prepare the output filename by appending "_reduced.fit"
    filename = os.path.join(output_directory, f"{original_filename}_reduced.fit")
    
    # Write out the reduced light frame to the reduced data directory
    reduced_light.write(filename, overwrite=True)


AttributeError: 'CCDData' object has no attribute 'filename'

In [ ]:
# Load the FITS file (replace 'your_image_file.fit' with your actual FITS file path)
fits_image = fits.open(output_directory + '/TB_M31_reduced.fit')

# Extract the data from the FITS file (typically in the first HDU)
image_data = fits_image[0].data

# Close the FITS file after extracting the data
fits_image.close()

# Normalize the image for better visualization (optional)
# You can try different intervals like PercentileInterval or ZScaleInterval
norm = ImageNormalize(image_data, interval=ZScaleInterval())

# Display the image using matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(image_data, cmap='gray', norm=norm)
plt.colorbar()
plt.title('FITS Image')
plt.show()


In [ ]:

# Load the FITS file (replace 'your_image_file.fit' with your actual FITS file path)
fits_image = fits.open('/home/idies/workspace/Temporary/treed28/scratch/20240903/TB_M31-001B.fit')

# Extract the data from the FITS file (typically in the first HDU)
image_data = fits_image[0].data

# Close the FITS file after extracting the data
fits_image.close()

# Normalize the image for better visualization (optional)
# You can try different intervals like PercentileInterval or ZScaleInterval
norm = ImageNormalize(image_data, interval=ZScaleInterval())

# Display the image using matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(image_data, cmap='gray', norm=norm)
plt.colorbar()
plt.title('FITS Image')
plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# from astropy.visualization import (PercentileInterval, ZScaleInterval, ImageNormalize)
# from astropy.io import fits

# Load the FITS file (replace 'your_image_file.fit' with your actual FITS file path)
fits_image = fits.open('/home/idies/workspace/Temporary/treed28/scratch/20240903/TB_M31-001V.fit')

# Extract the data from the FITS file (typically in the first HDU)
image_data = fits_image[0].data

# Close the FITS file after extracting the data
fits_image.close()

# Normalize the image for better visualization (optional)
# You can try different intervals like PercentileInterval or ZScaleInterval
norm = ImageNormalize(image_data, interval=ZScaleInterval())

# Display the image using matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(image_data, cmap='gray', norm=norm)
plt.colorbar()
plt.title('FITS Image')
plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# from astropy.visualization import (PercentileInterval, ZScaleInterval, ImageNormalize)
# from astropy.io import fits

# Load the FITS file (replace 'your_image_file.fit' with your actual FITS file path)
fits_image = fits.open('/home/idies/workspace/Temporary/treed28/scratch/20240903/TB_M31-001R.fit')

# Extract the data from the FITS file (typically in the first HDU)
image_data = fits_image[0].data

# Close the FITS file after extracting the data
fits_image.close()

# Normalize the image for better visualization (optional)
# You can try different intervals like PercentileInterval or ZScaleInterval
norm = ImageNormalize(image_data, interval=ZScaleInterval())

# Display the image using matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(image_data, cmap='gray', norm=norm)
plt.colorbar()
plt.title('FITS Image')
plt.show()


In [ ]:
#Corrected Cell for above Cell

# import matplotlib.pyplot as plt
# from astropy.visualization import (PercentileInterval, ZScaleInterval, ImageNormalize)
# from astropy.io import fits

# Load the FITS file (replace 'your_image_file.fit' with your actual FITS file path)
fits_image = fits.open('/home/idies/workspace/Temporary/treed28/scratch/20240903/reduced/TB_M31_reduced.fit')

# Extract the data from the FITS file (typically in the first HDU)
image_data = fits_image[0].data

# Close the FITS file after extracting the data
fits_image.close()

# Normalize the image for better visualization (optional)
# You can try different intervals like PercentileInterval or ZScaleInterval
norm = ImageNormalize(image_data, interval=ZScaleInterval())

# Display the image using matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(image_data, cmap='gray', norm=norm)
plt.colorbar()
plt.title('FITS Image')
plt.show()


In [ ]:
##Gain

# Main process
directory = '/home/idies/workspace/Temporary/treed28/scratch/20240903/'  # Directory containing bias FITS files

# Load the bias images (only those starting with "Bias" and ending with ".fit")
bias_images = f.load_fits_images(directory)

# Estimate the average readout noise
average_noise = f.readout_noise_from_bias_images(bias_images)

if average_noise is not None:
    print("Average Readout Noise (ADU):", average_noise)
else:
    print("Failed to calculate average readout noise.")


In [ ]:
import importlib
import Functions as f
importlib.reload(f)


# Main process
directory = '/home/idies/workspace/Temporary/treed28/scratch/20240903/'  # Directory containing flat and bias FITS files

# Estimate the average readout noise
#average_noise = f.readout_noise_from_bias_images(bias_images)

# Specify the filter type you want to load (e.g., "B.fit", "V.fit", "R.fit")
filter_type = "B.fit"  # Change this to "V.fit" or "R.fit" depending on the filter

# Load the flat field images with the specified filter type (those starting with "DomeFlat" and ending with the filter)
flat_images = f.load_fits_images_by_filter(directory, "DomeFlat", filter_type)

# Load the bias images (those starting with "Bias")
bias_images = f.load_fits_images(directory) #, "Bias", ".fit")

# Ensure that we have at least two flat field images and two bias images
if len(flat_images) >= 2 and len(bias_images) >= 2:
    # Use the first two flat images and the first two bias images to calculate gain and readout noise
    gain, readout_noise = f.calculate_gain_and_readout_noise(flat_images[0], flat_images[1], bias_images[0], bias_images[1])
    
    # Print the results
    print("Average Readout Noise (ADU):", average_noise)
    print(f"Gain (e-/ADU): {gain}")
    print(f"Readout Noise (e-): {readout_noise}")
else:
    print("Not enough flat field or bias images to perform calculations.")
